In [26]:
import langchain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI



In [18]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key=os.getenv("openai_api_key")

In [27]:
client=ChatOpenAI(api_key=api_key, model_name="gpt-3.5-turbo", temperature=0.5)

/home/saugat/Desktop/GenAI/GenAI_env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [52]:
RESPONSE_JSON={
    '1':{
        'mcq':'multiple choice questions',
        'options':{
            'a':'choice here',
            'b':'choice here',
            'c':'choice here',
            'c':'choice here',
            'd':''
        },
        'correct':'correct answer',
    },

    '3':{
        'mcq':'multiple choice questions',
        'options':{
            'a':'choice here',
            'b':'choice here',
            'c':'choice here',
            'c':'choice here',
            'd':''
        },
        'correct':'correct answer',
    },

    '3':{
        'mcq':'multiple choice questions',
        'options':{
            'a':'choice here',
            'b':'choice here',
            'c':'choice here',
            'c':'choice here',
            'd':''
        },
        'correct':'correct answer',
    },
    
}

In [61]:
Template1="""
text:{text}
you are a mcq generator.Based on the given text, your job is to generate {number} multiple choice question for
{subject} student in {tone} tone.Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure the format is in RESPONSE_JSON format which is given below and use it as a guide.
Enure to make {number} MCQs

{response_json}

"""

In [62]:
quiz_generation_prompt=PromptTemplate(
    input_variables=['text','number','subject','tone','response_json'],
    template=Template1
)

In [63]:
from langchain.chains import LLMChain
quiz_creation_chain=LLMChain(llm=client, prompt=quiz_generation_prompt,output_key='quiz',verbose=True)

In [64]:


Template2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""



In [65]:
quiz_evaluation_prompt=PromptTemplate(
    input_variables=['subject','quiz'],
    template=Template2
)

In [66]:
from langchain.chains import LLMChain
quiz_evaluation_chain=LLMChain(llm=client, prompt=quiz_evaluation_prompt,output_key="review",verbose=True)

In [67]:
from langchain.chains import SequentialChain

In [68]:
final_chain=SequentialChain(
    chains=[quiz_creation_chain,quiz_evaluation_chain],
    input_variables=['text','number','subject','tone','response_json'],
    output_variables=['quiz','review'],
    verbose=True
                            )



In [69]:
file_path="/home/saugat/Desktop/GenAI/data.txt"

In [70]:
with open(file_path,"r") as f:
    text=f.read()

In [71]:
text

'Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other scientists, 

In [72]:
RESPONSE_JSON

{'1': {'mcq': 'multiple choice questions',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': ''},
  'correct': 'correct answer'},
 '3': {'mcq': 'multiple choice questions',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': ''},
  'correct': 'correct answer'}}

In [73]:
number=5
subject="biology"
tone='simple'

In [74]:
from langchain.callbacks import get_openai_callback
import json

with get_openai_callback() as cb:
    response=final_chain(
        {
            'text':text,
            'number':number,
            "subject":subject,
            'tone':tone,
            "response_json":json.dumps(RESPONSE_JSON)
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

text:Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each define

In [75]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1286
Prompt Tokens:939
Completion Tokens:347
Total Cost:0.0021025


In [76]:
response

{'text': 'Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other sci

In [77]:
quiz=response.get('quiz')

In [78]:
quiz

'{\n"1": {\n"mcq": "What is the scientific study of life called?",\n"options": {\n"a": "Chemistry",\n"b": "Biology",\n"c": "Physics",\n"d": "Geology"\n},\n"correct": "b"\n},\n"2": {\n"mcq": "Which theme in biology explains the unity and diversity of life?",\n"options": {\n"a": "Evolution",\n"b": "Energy processing",\n"c": "Cell structure",\n"d": "Regulation of internal environments"\n},\n"correct": "a"\n},\n"3": {\n"mcq": "What allows organisms to move, grow, and reproduce?",\n"options": {\n"a": "Evolution",\n"b": "Energy processing",\n"c": "Cell structure",\n"d": "Regulation of internal environments"\n},\n"correct": "b"\n},\n"4": {\n"mcq": "What is the main unit of organization in biology?",\n"options": {\n"a": "Atoms",\n"b": "Molecules",\n"c": "Cells",\n"d": "Tissues"\n},\n"correct": "c"\n},\n"5": {\n"mcq": "When did life on Earth emerge?",\n"options": {\n"a": "1 million years ago",\n"b": "500 million years ago",\n"c": "3.7 billion years ago",\n"d": "10 billion years ago"\n},\n"corre

In [79]:
quiz=json.loads(quiz)

In [80]:
quiz

{'1': {'mcq': 'What is the scientific study of life called?',
  'options': {'a': 'Chemistry',
   'b': 'Biology',
   'c': 'Physics',
   'd': 'Geology'},
  'correct': 'b'},
 '2': {'mcq': 'Which theme in biology explains the unity and diversity of life?',
  'options': {'a': 'Evolution',
   'b': 'Energy processing',
   'c': 'Cell structure',
   'd': 'Regulation of internal environments'},
  'correct': 'a'},
 '3': {'mcq': 'What allows organisms to move, grow, and reproduce?',
  'options': {'a': 'Evolution',
   'b': 'Energy processing',
   'c': 'Cell structure',
   'd': 'Regulation of internal environments'},
  'correct': 'b'},
 '4': {'mcq': 'What is the main unit of organization in biology?',
  'options': {'a': 'Atoms', 'b': 'Molecules', 'c': 'Cells', 'd': 'Tissues'},
  'correct': 'c'},
 '5': {'mcq': 'When did life on Earth emerge?',
  'options': {'a': '1 million years ago',
   'b': '500 million years ago',
   'c': '3.7 billion years ago',
   'd': '10 billion years ago'},
  'correct': 'c'}}

In [82]:
quiz_data=[]

for key,value in quiz.items():
    mcq=value['mcq']
    options=" | ".join(
        [
             f"{option}: {option_value}"
            for option, option_value in value["options"].items()
        ]
    )
    correct_value=value['correct']
    quiz_data.append({
        "MCQ":mcq ,
        "CHOICES":options, 
        "CORRECT ANSWER":correct_value
                      })

In [83]:
quiz_data

[{'MCQ': 'What is the scientific study of life called?',
  'CHOICES': 'a: Chemistry | b: Biology | c: Physics | d: Geology',
  'CORRECT ANSWER': 'b'},
 {'MCQ': 'Which theme in biology explains the unity and diversity of life?',
  'CHOICES': 'a: Evolution | b: Energy processing | c: Cell structure | d: Regulation of internal environments',
  'CORRECT ANSWER': 'a'},
 {'MCQ': 'What allows organisms to move, grow, and reproduce?',
  'CHOICES': 'a: Evolution | b: Energy processing | c: Cell structure | d: Regulation of internal environments',
  'CORRECT ANSWER': 'b'},
 {'MCQ': 'What is the main unit of organization in biology?',
  'CHOICES': 'a: Atoms | b: Molecules | c: Cells | d: Tissues',
  'CORRECT ANSWER': 'c'},
 {'MCQ': 'When did life on Earth emerge?',
  'CHOICES': 'a: 1 million years ago | b: 500 million years ago | c: 3.7 billion years ago | d: 10 billion years ago',
  'CORRECT ANSWER': 'c'}]

In [85]:
import pandas as pd
quiz=pd.DataFrame(quiz_data)

In [86]:
quiz

,MCQ,CHOICES,CORRECT ANSWER
0,What is the scientific study of life called?,a: Chemistry | b: Biology | c: Physics | d: Ge...,b
1,Which theme in biology explains the unity and ...,a: Evolution | b: Energy processing | c: Cell ...,a
2,"What allows organisms to move, grow, and repro...",a: Evolution | b: Energy processing | c: Cell ...,b
3,What is the main unit of organization in biology?,a: Atoms | b: Molecules | c: Cells | d: Tissues,c
4,When did life on Earth emerge?,a: 1 million years ago | b: 500 million years ...,c


In [87]:
quiz.to_csv('MachineLearning.csv',index=False)